In [ ]:
# import libraries
import fastai
from fastai import *
from fastai.text import * 
import pandas as pd
import numpy as np
from functools import partial
import io
import os
from sklearn.model_selection import train_test_split


In [ ]:
data = pd.read_csv("category_features_TEST_1.csv") # REPLACE CSV FILE FOR TEST DATA 
df = pd.DataFrame({'label':data.label, 'text':data.text})
tester = df['text']
from sklearn.model_selection import train_test_split

# split data into training and validation set
df_trn_l, df_val_l = train_test_split(df, stratify = df['label'], test_size = 0.1, random_state = 12)
df_trn, df_val = train_test_split(df, stratify = df['label'], test_size = 0.3, random_state = 12)

#language model data - vectorization 
data_lm = TextLMDataBunch.from_df(train_df=df_trn, valid_df=df_val, path = "")

# Classifier model data - 
data_clas = TextClasDataBunch.from_df(path = "", train_df = df_trn, valid_df = df_val, vocab=data_lm.train_ds.vocab, bs=32)

learn = text_classifier_learner(data_clas, arch=AWD_LSTM, drop_mult=0.5)


In [ ]:
# learn.set_data(set_data([19072,400], 32))
learn = load_learner('models/', "classification_mod.pkl") #LOAD MODEL 

labels = df['label']
tester = df['text']

correct = 0
for i in range(len(tester)):
    cat, _, _ = learn.predict(tester[i])
    
    if str(cat) == labels[i]:
        correct+=1
        
print(correct/len(tester)*100)